In [55]:
import codecs
import re

In [88]:
f = codecs.open('example1.txt', 'r', 'utf-8')
text = f.readlines()
text = ''.join(text)
print(text)

00:17:02	Hassan:	morning, sir
00:18:02	Haidar Rahman (Edo):	Deals with Big Data
00:18:13	Habib Abdurrasyid:	networks that inspired by human neuron and how it works
00:18:14	Utih Amartiwi:	like a blackbox
00:18:40	Mutiara Ruci:	ann with 2 layer. deep learning have more layer?
00:18:56	Wayan Dadang - Frontera Orbit Class:	nonlinier model
00:24:26	Utih Amartiwi:	yes
00:24:28	Haidar Rahman (Edo):	yes
00:24:29	Pulung Hendro Prastyo:	clear
00:24:33	Farchan Hakim:	clear sir
00:24:36	Wayan Dadang - Frontera Orbit Class:	clear
00:24:38	Hassan:	yes, sir
00:31:04	Habib Abdurrasyid:	In data science domain, is deep learning rarely used sir?
00:31:52	Habib Abdurrasyid:	I see, thank you sir
00:34:08	otm3:	are there any good implementation of AI for image recognition which not using DL sir?
00:34:21	Dino Febriyanto:	sir, talk about mirror. with GAN we can "mirroring" someone face, right?
00:34:52	Dino Febriyanto:	ahh i see
00:34:57	Dino Febriyanto:	thanks sir
00:41:24	Habib Abdurrasyid:	Or maybe minin

In [89]:
timestamps = re.findall("\d{2}:\d{2}:\d{2}", text)
messages = re.split("\d{2}:\d{2}:\d{2}", text)
messages.pop(0)
users = []
comments = []
for message in messages:
    users.append(re.findall("\t([\S\s]+):", message)[0])
    comments.append(re.findall(":\t([\S\s]+)", message)[0].strip())

new_timestamps = []
for t in timestamps:
    second = int(t[-2:]) + 2
    new_timestamps.append(f"{t},000 --> {t[:-2]}{second},000")

In [97]:
with codecs.open('result.srt', 'w+', 'utf-8') as f:
    for i in range(len(users)):
        f.write(f"{str(i+1)}\n")
        f.write(f"{new_timestamps[i]}\n")
        f.write(f"{users[i]}: {comments[i]}\n\n")
    f.close()